# 서포트 벡터 머신(Support Vector Machines)

* 회귀, 분류, 이상치 탐지 등에 사용되는 지도학습 방법
* 클래스 사이의 경계에 위치한 데이터 포인트를 서포트 벡터(support vector)라고 함
* 각 지지 벡터가 클래스 사이의 결정 경계를 구분하는데 얼마나 중요한지를 학습
* 각 지지 벡터 사이의 마진이 가장 큰 방향으로 학습
* 지지 벡터 까지의 거리와 지지 벡터의 중요도를 기반으로 예측을 수행

![support vector machine](https://upload.wikimedia.org/wikipedia/commons/thumb/2/20/Svm_separating_hyperplanes.png/220px-Svm_separating_hyperplanes.png)

* H3은 두 클래스의 점들을 제대로 분류하고 있지 않음
* H1과 H2는 두 클래스의 점들을 분류하는데, H2가 H1보다 더 큰 마진을 갖고 분류하는 것을 확인할 수 있음

In [2]:
import multiprocessing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use(["seaborn-v0_8-whitegrid"])

In [6]:
from sklearn.svm import SVR, SVC
from sklearn.datasets import load_diabetes
from sklearn.datasets import load_breast_cancer, load_iris, load_wine
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.manifold import TSNE

## SVM을 이용한 회귀 모델과 분류 모델

### SVM을 사용한 회귀 모델 (SVR)

In [8]:
X, y = load_diabetes(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = SVR()
model.fit(X_train, y_train)
print(model.score(X_train, y_train))
print(model.score(X_test, y_test))


0.1919237405991968
0.08454650788981066


### SVM을 사용한 분류 모델 (SVC)

In [11]:
X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = SVC()
model.fit(X_train, y_train)
print(model.score(X_train, y_train))
print(model.score(X_test, y_test))


0.9208791208791208
0.8947368421052632


## 커널 기법

* 입력 데이터를 고차원 공간에 사상해서 비선형 특징을 학습할 수 있도록 확장하는 방법
* scikit-learn에서는 Linear, Polynomial, RBF(Radial Basis Function)등 다양한 커널 기법을 지원

![kernel trick](https://scikit-learn.org/stable/_images/sphx_glr_plot_iris_svc_0011.png)

In [13]:
X, y = load_diabetes(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

linear_svr = SVR(kernel="linear")
linear_svr.fit(X_train, y_train)
print(linear_svr.score(X_train, y_train))
print(linear_svr.score(X_test, y_test))

poly_svr = SVR(kernel="poly")
poly_svr.fit(X_train, y_train)
print(poly_svr.score(X_train, y_train))
print(poly_svr.score(X_test, y_test))

rbf_svr = SVR(kernel="rbf")
rbf_svr.fit(X_train, y_train)
print(rbf_svr.score(X_train, y_train))
print(rbf_svr.score(X_test, y_test))

0.0014260222139305112
0.00876770052619058
0.2744699907844378
0.31387999804601696
0.16527001525938645
0.18534074952226387


In [14]:
X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

linear_svc = SVC(kernel="linear")
linear_svc.fit(X_train, y_train)
print(linear_svc.score(X_train, y_train))
print(linear_svc.score(X_test, y_test))

poly_svc = SVC(kernel="poly")
poly_svc.fit(X_train, y_train)
print(poly_svc.score(X_train, y_train))
print(poly_svc.score(X_test, y_test))

rbf_svc = SVC(kernel="rbf")
rbf_svc.fit(X_train, y_train)
print(rbf_svc.score(X_train, y_train))
print(rbf_svc.score(X_test, y_test))


0.9626373626373627
0.956140350877193
0.9164835164835164
0.8947368421052632
0.9142857142857143
0.8859649122807017


## 매개변수 튜닝

* SVM은 사용하는 커널에 따라 다양한 매개변수 설정 가능
* 매개변수를 변경하면서 성능변화를 관찰

In [15]:
X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [16]:
poly_svc = SVC(kernel="poly", degree=2, C=0.1, gamma="auto")
poly_svc.fit(X_train, y_train)
print(poly_svc.score(X_train, y_train))
print(poly_svc.score(X_test, y_test))

0.9802197802197802
0.9210526315789473


In [18]:
rbf_svc = SVC(kernel="rbf", C=1.5, gamma="scale")
rbf_svc.fit(X_train, y_train)
print(rbf_svc.score(X_train, y_train))
print(rbf_svc.score(X_test, y_test))

0.9208791208791208
0.9122807017543859


## 데이터 전처리

* SVM은 입력 데이터가 정규화 되어야 좋은 성능을 보임
* 주로 모든 특성 값을 [0, 1] 범위로 맞추는 방법을 사용
* scikit-learn의 StandardScaler 또는 MinMaxScaler를 사용해 정규화

In [19]:
X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [20]:
model = SVC()
model.fit(X_train, y_train)
print(model.score(X_train, y_train))
print(model.score(X_test, y_test))


0.9252747252747253
0.8947368421052632


In [21]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [22]:
model = SVC()
model.fit(X_train, y_train)
print(model.score(X_train, y_train))
print(model.score(X_test, y_test))


0.9868131868131869
0.9736842105263158


In [23]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [24]:
model = SVC()
model.fit(X_train, y_train)
print(model.score(X_train, y_train))
print(model.score(X_test, y_test))


0.9846153846153847
0.9736842105263158


## Linear SVR

### 당뇨병